### Biostat 257: Homework 4

We are going to try different numerical methods learnt in class on the Google PageRank problem.

In [ ]:
using LinearAlgebra, CodecZlib, DelimitedFiles

In [122]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


### Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution  𝐱=(𝑥1,…,𝑥𝑛)𝑇  of the Markov chain.  𝑥𝑖  has the natural interpretation of the proportion of times the surfer visits page  𝑖  in the long run. Therefore  𝐱  serves as page ranks: a higher  𝑥𝑖  means page  𝑖  is more visited. It is well-known that  𝐱  is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix  𝐏 . That is  𝐏'𝐱=𝐱 . Therefore  𝐱  can be solved as an eigen-problem. It can also be cast as solving a linear system. Since the row sums of  𝐏  are 1,  𝐏  is rank deficient. We can replace the first equation by the $\Sigma x_i =1$ .

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix  𝐈−𝐏' directly if we start with a vector with all positive entries.

### Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package.

In [7]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: creating database file
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/download.jl:63
┌ Info: reading index files
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/download.jl:81
┌ Info: dowloading index file https://math.nist.gov/MatrixMarket/matrices.html
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/downloadmm.jl:83
┌ Warning: exception during initialization: 'ProcessFailedException(Base.Process[Process(`sh -c 'curl "https://math.nist.gov/MatrixMarket/matrices.html" -Lso "/Users/tinaarputhasamy/.julia/scratchspaces/b51810bb-c9f3-55da-ae3c-350fc1fbce05/data/mm_matrices.html"'`, ProcessExited(60))])'
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:147


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [8]:
# connectivity matrix
A = md.A

916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:

- How much memory does A take? If converted to a Matrix{Float64} (don't do it!), how much memory will it take?
- number of web pages
- number of edges (web links).
- number of dangling nodes (pages with no out links)
- histogram of in-degrees
- list the top 20 pages with the largest in-degrees?
- histogram of out-degrees
- which the top 20 pages with the largest out-degrees?
- visualize the sparsity pattern of  𝐀  or a submatrix of  𝐀  say A[1:10000, 1:10000].

Hint: For plots, you can use the `UnicodePlots.jl` package (spy, histogram, etc), which is fast for large data.

#### **Solution:**

##### - How much memory does A take? If converted to a Matrix{Float64} (don't do it!), how much memory will it take?

A takes about $916428^{2} \times 8$ gigabytes of RAM.

##### - Number of Webpages

The number of webpages is equal to either the number of rows or columns in the matrix (which are equal. `This is equal to: 916,428.`

In [101]:
# code to get dimensions 

size(A, 1)

916428

##### - Number of Edges

The number of edges (web links) is equal to the number of 1's in the matrix. `This is equal to: 5,105,039.`

In [103]:
# code to get number of 1's in A

count(i ->(i==1), A)

5105039

##### - Number of Dangling Nodes 

The number of dangling nodes (pages with no out links) is equivalent to the number of rows that are completely filled with zeros. This can be obtained by finding the number of row sums that equal 0. `This is equal to: 176,974.`

In [104]:
# code to get number of dangling nodes

rowsums = sum(A, dims = 2)
count(i ->(i==0), rowsums)

# sum(A::AbstractArray; dims)
# Sum elements of an array over the given dimensions
# set dims = 1 for column sums, set dims = 2 for row sums

176974

##### - Histogram of In-degrees

The number of in-degrees per page is equal to the column sums. 

In [105]:
using UnicodePlots

colsums = sum(A, dims = 1)
histogram(colsums)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤████████████████████████████████  916114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

##### - Top 20 pages with Largest In-degrees

In [106]:
colsums_ordered = sortperm(vec(colsums), rev= true) 
# returns indicies of sorted vector (rev = T because we want largest to smallest)
colsums_ordered[1:20] # extract top 20

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

##### - Histogram of Out-degrees

The number of out-degrees per page is equal to the row sums. 

In [107]:
using UnicodePlots

rowsums = sum(A, dims = 2)
histogram(rowsums)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤████████████████████████████████  891798  
   [ 20.0,  40.0) ┤▊ 22628                                   
   [ 40.0,  60.0) ┤▏ 1329                                    
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

##### - Top 20 pages with Largest Out-degrees

In [108]:
rowsums = sum(A, dims = 2)
rowsums_ordered = sortperm(vec(rowsums), rev = true)
rowsums_ordered[1:20]

20-element Vector{Int64}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

##### - Visualize the sparsity pattern of 𝐀 or a submatrix of 𝐀 say A[1:10000, 1:10000]

In [60]:
spy(A) # entire matrix

          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
          ┌──────────────────────────────────────────┐    
        1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│  

In [61]:
spy(A[1:10000, 1:10000]) # submatrix A

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

### Q4 (5 pts) Dense linear algebra?

Consider the following methods to obtain the page ranks of the SNAP/web-Google data.

1. A dense linear system solver such as LU decomposition.
2. A dense eigen-solver for asymmetric matrix.

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer.

**1. Memory Usage Estimate:**

$\frac{2}{3} * (916428)^3 + 2*(916428)^2 = 5.13 x 10^{17}$ total flops

**2. Theoretical Throughput:**

Intel Skylake CPUs (check my versioninfo()) can do 16 DP flops per cycle and 32 SP flops per cycle. Therefore,
the theoretical throughput of my laptop is: 

$4×2.9×109×16=185.6$ GFLOPS DP in double-precision, and $4×2.9×109×32=371.2$ GFLOPS SP, in single-precision. Using the Linear Algebra package within Julia, the peak flop rate of my computer is: $1.458003904777138 x 10^{11}$ flops per second.

Therefore, in double-precision, it would take my laptop: 

$5.13 x 10^{17}$ /$1.458003904777138 x 10^{11}$  = 3519221 seconds for my laptop use the LU approach (assuming that the LAPACK functions can achieve the theoretical throughput of my laptop). This is equivalent to roughly 41 days. 

### Q5 (75 pts) Iterative solvers

Set the teleportation parameter at  𝑝=0.85 . Consider the following methods for solving the PageRank problem.

1. An iterative linear system solver such as GMRES.
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this page. The start-up code below uses the KrylovKit.jl package. You can use other packages if you prefer. Make sure to utilize the special structure of  𝐏  (sparse + rank 1) to speed up the matrix-vector multiplication.

#### **Solution:**

#### Step 1 (15 pts)

Let's implement a type PageRankImPt that mimics the matrix  𝐌=𝐈−𝐏' . For iterative methods, all we need to provide are methods for evaluating  𝐌𝐯  and  𝐌𝑇𝐯  for arbitrary vector  𝐯 .

In [71]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    od        :: Vector{T} # nx1 storage for row sums (outdegrees)
    storage_d :: Vector{T} # nx1 storage for di's
    storage_a :: Vector{T} # nx1 storage for (1-p)/n 
    val1      :: Vector{T} # nx1 storage for intermediate calc 
    val2      :: Vector{T} # nx1 storage for intermediate calc 
    val3      :: Vector{T} # nx1 storage for intermediate calc 
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    #initialize
    n = size(A, 1) 
    od = A*ones(n) # calculating row sums (outdegrees) 
    # sum(A, dims = 2) doesn't seem to work because output is of type matrix...
    storage_a = zeros(n) # initializing with zero vector
    storage_d = similar(storage_a) 
    val1 = similar(storage_a)
    val2 = similar(storage_a)
    val3 = similar(storage_a)
    
    #make P
    for i in 1:n
        if od[i] == 0
            storage_d[i] = 0 
            storage_a[i] = 1/n
        else
            storage_d[i] = telep/od[i] 
            storage_a[i] = (1 - telep)/n
        end
    end

    PageRankImPt(A, telep, od, storage_d, storage_a, val1, val2, val3)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

# TODO: implement this function for evaluating M[i, j]

Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.storage_d[j] * M.A[j, i] - M.storage_a[j] 
# gets diagonals not sure why we need this 
# (also shouldn't diagonal elements have a 1 minus in front?)

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    
    # P' = I - A'diag(d) - 1na' 
    # (I-P')v = v - A'diag(d)v - 1na'v 
    
    # term 1 (A'diag(d)v)
    M.val1 .= M.storage_d .* v # diag(d) * v using elementwise mult
    mul!(M.val2, transpose(M.A), M.val1) # A'*(diag(d) * v)
    
    # term 2 (1na'v)
    scalar = dot(M.storage_a, v) 
    # dot product gives scalar, don't need to preallocate in constructor 
    
    # final product ((I-P')v)
    out .= v .- M.val2 .- scalar
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    
    # P = I - diag(d)A - n1n'
    # (I-P)v = v - diag(d)Av - n1n'v
    
    # term 1 (diag(d)Av)
    mul!(M.val1, M.A, v) # getting A*v first bc want to do elementwise mult with diag(d)
    M.val2 .= M.storage_d .* M.val1 # diag(d) * A * v (using elementwise mult)  
    
    # term 2 (n1n'v)
    M.val3 .= M.storage_a .* sum(v)
    
    # final product ((I-P)v)
    out .= v .- M.val2 .- M.val3
end

In [56]:
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

In [65]:
M = PageRankImPt(A, 0.85) # note! this is not the same M = I - P^T
n = size(M, 1)

916428

In [72]:
@assert norm(transpose(M) * ones(n)) < 1e-13

In [73]:
@assert norm(M * xsol) < 1e-13

#### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [42]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 97 samples with 1 evaluation.
 Range (min … max):  44.153 ms … 78.582 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     46.942 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.566 ms ±  8.456 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▃█                                                         
  ▆█████▇▆▅▅▃▅▃▁▃▁▃▁▃▁▃▁▁▁▄▁▁▁▁▃▃▁▁▁▄▁▃▃▁▁▁▁▃▁▃▁▁▃▁▁▃▁▁▁▃▁▄▁▃ ▁
  44.2 ms         Histogram: frequency by time        76.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [47]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):   66.922 ms … 195.716 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     105.946 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   112.660 ms ±  41.734 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▆▃▅▁▃▁▁▃▁▃▁▁▁▃▃▁▁▅▁▃▁▁▃▁▁▁▁▁▁▃▃▃▁▁▅▁▃▃▃▅▃▅▃▃▁▁▁▁▁▁▃▁▁▁▁▅▁▁▁▃ ▁
  66.9 ms          Histogram: frequency by time          196 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [48]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

#### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations.

In [74]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.3783428221986054e-5, 2.0710155392544003e-6, 3.6630659852478028e-6, 7.527510785625115e-7, 8.633285997186966e-7, 1.7694182527430189e-6, 2.4312303829196376e-7, 6.368417180766123e-7, 4.744973703764497e-7, 2.689548611104835e-7  …  3.1679469739868288e-6, 9.726880410259511e-7, 6.588614478562195e-7, 7.737011774737646e-7, 2.4312303829196376e-7, 1.6219204214293997e-6, 3.9121300606432984e-7, 2.4312303829196376e-7, 7.2960338313468936e-6, 6.330939996697664e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821151654716082e-13,).
), time = 8.529003491, bytes = 1009314489, gctime = 0.350002933, gcstats = Base.GC_Diff(1009314489, 138, 0, 79005, 4, 0, 350002933, 1, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [78]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is:

In [79]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

#### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems.

In [81]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-3.653836029264281e-14 + 0.0im], Vector{ComplexF64}[[0.005635826953807308 + 0.0im, 0.00034549143807828384 + 0.0im, 0.000611080849412529 + 0.0im, 0.00012557561626036273 + 0.0im, 0.00014402240532804286 + 0.0im, 0.0002951783050398634 + 0.0im, 4.055832828678566e-5 + 0.0im, 0.00010623935784852789 + 0.0im, 7.915671116196516e-5 + 0.0im, 4.4867650667256554e-5 + 0.0im  …  0.0005284839899796966 + 0.0im, 0.0001622659914798883 + 0.0im, 0.00010991273837610668 + 0.0im, 0.00012907055855752622 + 0.0im, 4.055832828678567e-5 + 0.0im, 0.00027057238743205243 + 0.0im, 6.526302748334149e-5 + 0.0im, 4.055832828678567e-5 + 0.0im, 0.001217140660132936 + 0.0im, 0.00010561415510700478 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.575262040976209e-14,).
), time = 19.327307858, bytes = 1581385541, gctime = 0.275003826, gcstats = Base.GC_Diff(1581385541, 213, 0, 77842, 20, 0, 275003826, 5, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError.`**

In [82]:
@assert abs(Real(vals[1])) < 1e-8

In [83]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get are:

In [85]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

### Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees?

#### **Solution**: 

From the results below we see that the top twenty page results from GRMES are the same as the top twenty page results obtained from using the Arnoldi method. However, it is also evident that the top twenty indegrees are NOT the same as the top twenty page results obtained using GRMES or the Arnoldi method. 

In [ ]:
# top twenty page results from grmes

In [112]:
page_ranks_g = sortperm(x_gmres, rev = true) # using gmres method
top_twenty_grmes = page_ranks[1:20]

20-element Vector{Int64}:
 597622
  41910
 163076
 537040
 384667
 504141
 486981
 605857
  32164
 558792
 551830
 765335
 751385
 425771
 908352
 173977
   7315
 213433
 885606
 819224

In [ ]:
# top twenty page results from arnoldi

In [110]:
page_ranks_a = sortperm(x_arnoldi, rev = true) # using arnoldi method
top_twenty_arnoldi = page_ranks[1:20]

20-element Vector{Int64}:
 597622
  41910
 163076
 537040
 384667
 504141
 486981
 605857
  32164
 558792
 551830
 765335
 751385
 425771
 908352
 173977
   7315
 213433
 885606
 819224

In [ ]:
# top twenty pages with respect to indegrees 

In [114]:
colsums_ordered = sortperm(vec(colsums), rev= true) 
# returns indicies of sorted vector (rev = T because we want largest to smallest)
top_twenty_indegrees = colsums_ordered[1:20] # extract top 20

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

In [117]:
top_twenty_arnoldi == top_twenty_grmes# is arnoldi same as grmes

true

In [118]:
top_twenty_arnoldi == top_twenty_indegrees # is arnoldi same as indegrees

false

In [119]:
top_twenty_grmes == top_twenty_indegrees # is grmes same as indegrees

false